In [ ]:
# -*- coding: utf-8 -*-
# ---
# jupyter:
#   jupytext:
#     text_representation:
#       extension: .py
#       format_name: light
#       format_version: '1.5'
#       jupytext_version: 1.14.0
#   kernelspec:
#     display_name: Python 3 (ipykernel)
#     language: python
#     name: python3
# ---

# # 6. Pipeline Xử lý Hoàn chỉnh

# Mục tiêu: Tích hợp các bước đã thử nghiệm thành một quy trình xử lý đầy đủ và trực quan.

# ## Import thư viện
import os
import sys
import cv2
import numpy as np

# Thêm thư mục src vào sys.path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src import utils, enhancement, restoration, segmentation, morphology



In [ ]:
# ## Tải ảnh mẫu
image_filename = 'sample_sign.jpg' # Đảm bảo ảnh này tồn tại
image_path = os.path.join('..', 'data', 'raw', image_filename) 

img_original = utils.load_image(image_path, cv2.IMREAD_COLOR)

if img_original is not None:
    utils.display_image("1. Ảnh Gốc", img_original)

    # ## Bước 2: Tăng cường Ảnh
    #img_enhanced = enhancement.histogram_equalization(img_original)
    img_enhanced = enhancement.contrast_stretching(img_original) # Thường hiệu quả hơn cho màu
    utils.display_image("2. Ảnh sau Tăng cường (Giãn Tương phản)", img_enhanced)

    # ## Bước 3: Khôi phục Ảnh
    # Làm mịn nhẹ để giảm nhiễu trước khi phân vùng
    img_restored = restoration.apply_median_filter(img_enhanced, kernel_size=3) 
    #img_restored = restoration.apply_gaussian_blur(img_enhanced, kernel_size=(3,3))
    utils.display_image("3. Ảnh sau Khôi phục (Lọc Median)", img_restored)

    # ## Bước 4: Phân vùng Ảnh
    # Kết hợp màu đỏ và xanh dương
    lower_red1 = np.array([0, 100, 100]); upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([160, 100, 100]); upper_red2 = np.array([180, 255, 255])
    lower_blue = np.array([100, 150, 50]); upper_blue = np.array([140, 255, 255])

    mask_red1 = segmentation.color_threshold_segmentation(img_restored, lower_red1, upper_red1)
    mask_red2 = segmentation.color_threshold_segmentation(img_restored, lower_red2, upper_red2)
    mask_blue = segmentation.color_threshold_segmentation(img_restored, lower_blue, upper_blue)
    
    mask_segmented = cv2.bitwise_or(mask_red1, mask_red2)
    mask_segmented = cv2.bitwise_or(mask_segmented, mask_blue)
    utils.display_image("4. Mask Phân vùng (Đỏ hoặc Xanh)", mask_segmented)

    # ## Bước 5: Xử lý Hình thái
    # Loại bỏ nhiễu nhỏ bằng phép Mở
    mask_opened = morphology.apply_opening(mask_segmented, kernel_size=(3, 3))
    # Lấp đầy lỗ hổng bằng phép Đóng
    mask_final = morphology.apply_closing(mask_opened, kernel_size=(7, 7))
    utils.display_image("5. Mask sau Xử lý Hình thái (Mở -> Đóng)", mask_final)

    # ## Bước 6: Trích xuất và Hiển thị Kết quả
    # Tìm contours
    contours, _ = cv2.findContours(mask_final, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    img_result = img_original.copy()
    
    if contours:
        min_contour_area = 500 # Điều chỉnh ngưỡng diện tích này
        valid_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_contour_area]
        
        if valid_contours:
            # Vẽ tất cả các contour hợp lệ tìm được
            cv2.drawContours(img_result, valid_contours, -1, (0, 255, 0), 2) 
            
            # Hoặc chỉ vẽ contour lớn nhất
            # largest_contour = max(valid_contours, key=cv2.contourArea)
            # x, y, w, h = cv2.boundingRect(largest_contour)
            # cv2.rectangle(img_result, (x, y), (x + w, y + h), (0, 255, 0), 3) 
            
            print(f"Tìm thấy {len(valid_contours)} đối tượng tiềm năng.")
        else:
            print("Không tìm thấy đối tượng đủ lớn.")
    else:
        print("Không tìm thấy đối tượng nào.")

    utils.display_image("6. Kết quả Cuối cùng", img_result)

else:
    print(f"Lỗi: Không thể tải ảnh tại '{image_path}'.")